In [1]:
import pandas as pd
import numpy as np
import string
import math
from collections import Counter

In [2]:
# Read emotions from the file
filepath = "NRC-Emotions-Lexicon\NRC-Emotion-Lexicon-v0.92\NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emotions = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')

In [8]:
# Convert emotions data to new dataframe
emotions_words = emotions.pivot(index='word', columns='emotion', values='association').reset_index()
emotions_words.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abandonment,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0
2,abate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,abatement,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abba,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [9]:
def clean_review(r):
    return r.replace("\n", " ").translate(None, string.punctuation).split()

def fill_emotions(r):
    tmp = emotions_words[emotions_words.isin({'word': r}).word]
    # get the frequency of words in the review and return words that repeated more than one in the review and appered in emotions dataset
    freq_words=[(k,v) for k,v in Counter(r).items() if v>1 and k in tmp.word.values]
    for k,v in freq_words:
        tmp=tmp.append([tmp[tmp.word==k]]*v,ignore_index=True)#Less efficient: the More efficient: in this link https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html
    #print (tmp[tmp.columns[2:len(tmp)]].sum(axis=0))
    #print (tmp)
    return pd.DataFrame(tmp.mean()).T
    #tmp = tmp.replace(0, np.NaN)
    #print (tmp.mean())
    
def get_emotions(rgrp):
    lst=[]
    for r in rgrp:
        lst.append(fill_emotions(clean_review(r)))
    result = pd.concat(lst,ignore_index=True) #ignore_index=True, sort=False
    return pd.DataFrame(result.mean()).T
    return fill_emotions(clean_review(rgrp.iloc[0]))

#-------------Main----------------------
test = pd.read_csv("test.csv")
Gtest=test.groupby('title')
lst=[]
for grp in Gtest.groups.keys():
    tmp=get_emotions(Gtest.get_group(grp)['review'])
    tmp['title']=grp
    lst.append(tmp)
result = pd.concat(lst,ignore_index=True)
result

#Gtest.get_group('Toy Story,(1995)')['review']
#final=get_emotions(Gtest.get_group('Toy Story,(1995)')['review'])
#final

emotion,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,title
0,0.029868,0.143792,0.023003,0.041013,0.165365,0.073402,0.291321,0.033169,0.081865,0.163447,"Toy Story,(1995)"
1,0.048190,0.153632,0.022942,0.049318,0.160662,0.089198,0.253463,0.055585,0.082530,0.160529,"Grumpier Old Men,(1995)"
2,0.030403,0.212220,0.027201,0.066704,0.219045,0.100927,0.299222,0.065344,0.110649,0.187320,"Father of the Bride Part II,(1995)"
3,0.079598,0.145258,0.075624,0.059740,0.135867,0.158551,0.245801,0.093687,0.069552,0.135718,"Waiting to Exhale,(1995)"
4,0.034288,0.152919,0.021671,0.070661,0.149051,0.084283,0.235161,0.035117,0.061304,0.120883,"Jumanji,(1995)"


In [10]:
review1 = open('review.txt')
review1 = review1.read()
review1 = review1.replace("\n", " ")
review1 = review1.translate(None, string.punctuation)
review1 = review1.split()
#review1

# review2 = open('review_2.txt')
# review2 = review2.read()
# review2 = review2.replace("\n", " ")
# review2 = review2.translate(None, string.punctuation)
# review2 = review2.split()

# toystory = open('toystory_screen.txt')
# toystory = toystory.read()
# screen = [toystory]

In [12]:
def movie_profile(reviews):
    emotions = {'anger':[],
                'anticipation':[],
                'disgust':[],
                'fear':[],
                'joy':[],
                'negative':[],
                'positive':[],
                'sadness':[],
                'surprise':[],
                'trust':[]}

    review_count = len(reviews)

    i = 0       
    for review in reviews:
        review = review.replace("\n", " ")
        review = review.translate(None, string.punctuation)
        review = review.split()

        for emotion in emotions:
            emotions[emotion].append(0)

        for word in review:
            if ((emotions_words.word == word).any()):
                word_emotion = emotions_words[emotions_words.word == word]    
                for emotion in emotions:
                    emotions[emotion][i] += word_emotion[emotion].item()

        for emotion in emotions:
            emotions[emotion][i] /= len(review)
        i += 1

    for emotion in emotions:
        emotions[emotion].append(np.mean(emotions[emotion]))
        del emotions[emotion][0:review_count]

    emotions_df = pd.DataFrame.from_dict(emotions)
    return emotions_df

In [14]:
test = pd.read_csv("test.csv")
reviews = test.pivot(index='title', columns='review_number', values='review')

movies=[]
for title, review in reviews.iterrows():
    movies.append(review)

def delete_nan(reviews):
    count = len(reviews)
    for i in range(count):
        if(isinstance(reviews[i+1], float)):
            del reviews[i+1]

   
delete_nan(movies[3])
movie_profile(movies[3])


# movie_profile(screen)

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,0.006155,0.03025,0.005023,0.008609,0.034524,0.015831,0.06273,0.007185,0.017356,0.034782


In [890]:
delete_nan(review])
movie_profile(movies[3])

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,0.006155,0.03025,0.005023,0.008609,0.034524,0.015831,0.06273,0.007185,0.017356,0.034782
